In [ ]:
"""
Enhanced VGG16 Visualization for Understanding Convolutional Neural Networks
This notebook demonstrates how CNNs learn hierarchical features from images.
"""

import os
import numpy as np
import matplotlib.pyplot as plt
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.preprocessing.image import load_img, img_to_array
from keras.models import Model

In [ ]:
# Set matplotlib style for better visualizations
plt.style.use('seaborn-v0_8-darkgrid')

In [ ]:
# ============================================================================
# SECTION 1: Load and Prepare the VGG16 Model
# ============================================================================

def load_vgg16_model():
    """
    Load the pre-trained VGG16 model.
    VGG16 is a 16-layer deep CNN trained on ImageNet dataset.
    It learns to recognize 1000 different object categories.
    """
    print("Loading VGG16 model (this may take a moment)...")
    model = VGG16(weights='imagenet')
    print(f"Model loaded successfully! Total layers: {len(model.layers)}")
    return model

In [ ]:
full_model = load_vgg16_model()

# Display model architecture summary
print("\n" + "="*70)
print("VGG16 ARCHITECTURE SUMMARY")
print("="*70)
full_model.summary()

In [ ]:
# ============================================================================
# SECTION 2: Understanding the Convolutional Layers
# ============================================================================

def analyze_conv_layers(model):
    """
    Analyze and display information about all convolutional layers.
    This helps us understand the architecture of the network.
    """
    print("\n" + "="*70)
    print("CONVOLUTIONAL LAYERS ANALYSIS")
    print("="*70)
    
    conv_layers = []
    for idx, layer in enumerate(model.layers):
        if 'conv' in layer.name:
            filters, biases = layer.get_weights()
            conv_layers.append({
                'index': idx,
                'name': layer.name,
                'filters_shape': filters.shape,
                'num_filters': filters.shape[3]
            })
            print(f"Layer {idx}: {layer.name}")
            print(f"  Filter shape: {filters.shape}")
            print(f"  Number of filters: {filters.shape[3]}")
            print(f"  Interpretation: This layer has {filters.shape[3]} different ")
            print(f"                  feature detectors, each looking for specific patterns\n")
    
    return conv_layers

conv_layer_info = analyze_conv_layers(full_model)

In [ ]:
# ============================================================================
# SECTION 3: Load and Preprocess Input Image
# ============================================================================

def load_and_preprocess_image(image_path, target_size=(224, 224)):
    """
    Load an image and prepare it for VGG16 processing.
    
    Args:
        image_path: Path to the input image
        target_size: Size to resize image (VGG16 requires 224x224)
    
    Returns:
        Preprocessed image ready for model input
    """
    # Check if file exists
    if not os.path.exists(image_path):
        raise FileNotFoundError(
            f"Image file '{image_path}' not found. "
            f"Please provide a valid image path."
        )
    
    # Load and resize image
    img = load_img(image_path, target_size=target_size)
    
    # Convert to array and add batch dimension
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    
    # Preprocess for VGG16 (subtract ImageNet mean)
    img_preprocessed = preprocess_input(img_array)
    
    return img, img_preprocessed

# Try to load the image with error handling
try:
    original_img, processed_img = load_and_preprocess_image('me.jpg')
    print("\n" + "="*70)
    print("IMAGE LOADED SUCCESSFULLY")
    print("="*70)
    print(f"Original image size: {original_img.size}")
    print(f"Preprocessed array shape: {processed_img.shape}")
    
    # Display the original image
    plt.figure(figsize=(6, 6))
    plt.imshow(original_img)
    plt.title("Original Input Image", fontsize=14, fontweight='bold')
    plt.axis('off')
    plt.tight_layout()
    plt.savefig('01_original_image.png', dpi=150, bbox_inches='tight')
    plt.show()
    
except FileNotFoundError as e:
    print(f"\nError: {e}")
    print("Please ensure you have an image named 'me.jpg' in the working directory.")
    print("You can change the filename in the code to match your image.")


In [ ]:
# ============================================================================
# SECTION 4: Visualize First Layer Filters (What the Network Looks For)
# ============================================================================

def visualize_first_layer_filters(model, num_filters=8):
    """
    Visualize the learned filters in the first convolutional layer.
    These filters detect basic features like edges, colors, and textures.
    
    The first layer learns simple patterns because it sees raw pixel values.
    """
    print("\n" + "="*70)
    print("FIRST LAYER FILTER VISUALIZATION")
    print("="*70)
    
    # Get filters from the first convolutional layer
    filters, biases = model.layers[1].get_weights()
    
    # Normalize filter values to [0, 1] for visualization
    f_min, f_max = filters.min(), filters.max()
    filters_normalized = (filters - f_min) / (f_max - f_min)
    
    print(f"Visualizing {num_filters} filters from the first conv layer")
    print(f"Each filter has 3 channels (R, G, B)")
    print(f"These filters detect basic features like edges and color gradients\n")
   
    # Create visualization
    fig = plt.figure(figsize=(15, num_filters * 1.5))
    
    for i in range(num_filters):
        f = filters_normalized[:, :, :, i]
        
        # Display each color channel separately
        for j in range(3):
            ax = plt.subplot(num_filters, 3, i * 3 + j + 1)
            ax.set_xticks([])
            ax.set_yticks([])
            
            # Add labels for the first row
            if i == 0:
                channel_names = ['Red Channel', 'Green Channel', 'Blue Channel']
                ax.set_title(channel_names[j], fontsize=10, fontweight='bold')
            
            # Add filter number label
            if j == 0:
                ax.set_ylabel(f'Filter {i+1}', fontsize=10, fontweight='bold')
            
            plt.imshow(f[:, :, j], cmap='gray')
    
    plt.suptitle('First Layer Convolutional Filters (Learned Feature Detectors)', 
                 fontsize=14, fontweight='bold', y=0.995)
    plt.tight_layout()
    plt.savefig('02_first_layer_filters.png', dpi=150, bbox_inches='tight')
    plt.show()
    
    return filters_normalized
try:
    first_layer_filters = visualize_first_layer_filters(full_model, num_filters=8)
except Exception as e:
    print(f"Error visualizing filters: {e}")

In [ ]:
# ============================================================================
# SECTION 5: Extract and Visualize Feature Maps from Multiple Layers
# ============================================================================

def create_feature_extractor(model, layer_name):
    """
    Create a model that outputs feature maps from a specific layer.
    This allows us to see what the network "sees" at different depths.
    """
    layer_output = model.get_layer(layer_name).output
    return Model(inputs=model.input, outputs=layer_output)

def visualize_feature_maps(feature_maps, layer_name, num_features=64, grid_size=8):
    """
    Visualize the feature maps (activations) from a convolutional layer.
    
    Feature maps show which parts of the image activate each filter.
    - Bright areas: Strong activation (filter detected its pattern)
    - Dark areas: Weak activation (pattern not present)
    """
    print(f"\nVisualizing {num_features} feature maps from {layer_name}")
    print(f"Feature maps shape: {feature_maps.shape}")
    
    # Create figure with proper size
    fig = plt.figure(figsize=(15, 15))
    
    for i in range(num_features):
        ax = plt.subplot(grid_size, grid_size, i + 1)
        ax.set_xticks([])
        ax.set_yticks([])
        
        # Display the feature map
        plt.imshow(feature_maps[0, :, :, i], cmap='viridis')
    
    plt.suptitle(f'Feature Maps from {layer_name}\n' + 
                 'Bright areas show where the filter detected its pattern',
                 fontsize=14, fontweight='bold')
    plt.tight_layout()
    
    # Save with a filename based on layer name
    filename = f'03_feature_maps_{layer_name}.png'
    plt.savefig(filename, dpi=150, bbox_inches='tight')
    plt.show()

# Visualize feature maps from multiple layers to show hierarchical learning
try:
    layers_to_visualize = [
        ('block1_conv1', 64, 8),  # Early layer: edges, colors
        ('block2_conv1', 64, 8),  # Middle layer: textures, simple shapes
        ('block3_conv1', 64, 8),  # Deeper layer: complex patterns
    ]
    
    print("\n" + "="*70)
    print("FEATURE MAPS VISUALIZATION")
    print("="*70)
    print("Feature maps show what each filter 'sees' in the image.")
    print("Early layers detect simple features, deeper layers detect complex patterns.\n")
    
    for layer_name, num_features, grid_size in layers_to_visualize:
        # Create feature extractor for this layer
        feature_extractor = create_feature_extractor(full_model, layer_name)
        
        # Extract features
        features = feature_extractor.predict(processed_img, verbose=0)
        
        # Visualize
        visualize_feature_maps(features, layer_name, num_features, grid_size)
        
except Exception as e:
    print(f"Error during feature extraction: {e}")

In [ ]:
# ============================================================================
# SECTION 6: Compare Multiple Layers Side by Side
# ============================================================================

def compare_layer_depths(model, image, layers=['block1_conv1', 'block3_conv1', 'block5_conv1']):
    """
    Compare how different depth layers respond to the same image.
    This demonstrates the hierarchical nature of CNN feature learning.
    """
    print("\n" + "="*70)
    print("HIERARCHICAL FEATURE LEARNING COMPARISON")
    print("="*70)
    
    fig, axes = plt.subplots(1, len(layers) + 1, figsize=(20, 4))
    
    # Show original image
    axes[0].imshow(original_img)
    axes[0].set_title('Original Image', fontsize=12, fontweight='bold')
    axes[0].axis('off')
    
    # Show feature maps from each layer
    for idx, layer_name in enumerate(layers):
        extractor = create_feature_extractor(model, layer_name)
        features = extractor.predict(image, verbose=0)
        
        # Show the first feature map from each layer
        axes[idx + 1].imshow(features[0, :, :, 0], cmap='viridis')
        axes[idx + 1].set_title(f'{layer_name}\n(Depth: Layer {idx + 1})', 
                                fontsize=12, fontweight='bold')
        axes[idx + 1].axis('off')
    
    plt.suptitle('How CNN Sees the Image at Different Depths\n' +
                 'Notice how features become more abstract in deeper layers',
                 fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.savefig('04_hierarchical_comparison.png', dpi=150, bbox_inches='tight')
    plt.show()
    
    print("\nInterpretation:")
    print("- Block 1: Detects edges, colors, and basic textures")
    print("- Block 3: Detects more complex shapes and patterns")
    print("- Block 5: Detects high-level features and object parts")

try:
    compare_layer_depths(full_model, processed_img)
except Exception as e:
    print(f"Error in comparison: {e}")

In [ ]:
# ============================================================================
# SECTION 7: Summary and Key Takeaways
# ============================================================================

print("\n" + "="*70)
print("KEY TAKEAWAYS ABOUT CNNs")
print("="*70)
print("""
1. HIERARCHICAL LEARNING: CNNs learn features in a hierarchy
   - Early layers: Simple features (edges, colors)
   - Middle layers: Textures and patterns
   - Deep layers: Complex objects and concepts

2. FILTERS ARE FEATURE DETECTORS: Each filter learns to detect a specific pattern
   - The network automatically learns these patterns from data
   - No manual feature engineering needed

3. FEATURE MAPS SHOW ACTIVATION: Bright areas in feature maps indicate
   where the filter found its pattern in the image

4. TRANSLATION INVARIANCE: CNNs can detect features regardless of their
   position in the image due to the convolutional structure

5. PARAMETER SHARING: The same filter is applied across the entire image,
   making CNNs efficient and good at generalization
""")